# Fitting parameters with SciPy

## Logistic Regression

In [ ]:
import numpy as np
from scipy import optimize
from matplotlib import pyplot as plt
%matplotlib inline 

In this example, a logistic function will be fitted to synthetic data using scipy's ``optimize`` function.

The logistic function is defined as
$$ f(x) = \frac{a}{1+\exp\big(-(b x - x_0)\big)} ,$$

where $a$ is the maximum value of the $f(x)$, $b$ determines the steepness of the curve, and $x_0$ is the midpoint of the curve, which is also the point of half-maximum and the inflection point.

First, we'll code the logistic function in python:

In [ ]:
def logistic(params,x):
    """Logistic function

    Parameters
    ----------
    params : list or numpy array
      the three parameters of the logistic function
      
    x : numpy array
      the explanatory variable
   
    Return
    ------
    numpy array
      the output of the logistic function

    """
    return params[0]/(1+np.exp(-x*params[1] - params[2])) 

When $a=b=x_{0}=1$, the logistic function looks like this:

In [ ]:
x = np.linspace(-8,8,100)          # creating a set of x values
y = logistic([1,1,1],x)            # parsing the x values through the logistic function

# Plotting
plt.plot(x, y)
plt.xlabel('x', size=16)
plt.ylabel('y', size=16)
plt.show()   

Note how the y lies between 0 and 1. Owing to the way we've coded it, the logistic function cannot be negative.

Now, let's create some synthetic data to fit to. We'll set $a=3$, $b=2$, and $x_0=0$ and add some noise to the data. The noise will be sampled from a Gaussian distribution with a mean of 0 and standard deviation of 0.05.

In [ ]:
x = np.linspace(-8,8,50)          
y = logistic([3,2,0],x)  + np.random.normal(0,0.1,size=50)

# Having a look at what that looks like:
plt.scatter(x, y)
plt.xlabel('x', size=16)
plt.ylabel('y', size=16)
plt.title('Synthetic data')
plt.show()   

To fit, we'll create a function that returns the residuals of our trial fit to the noisy data. When fitting the parameters, we'll be minimising the residuals, which in this case is the sum of squares.

In [ ]:
def residuals(params):
    predicted = logistic(params,x)
    return np.sum((y-predicted)**2)

Starting from an initial guess of the parameters, will minimise the residuals with SciPy's `optimize` function.

In [ ]:
initial_guess = [1,1,1]
fit = optimize.minimize(residuals,initial_guess,method='BFGS')
print "The predicted parameters are", fit.x

These look very close to the true parameters of the model:  $a=3$, $b=2$, and $x_0=0$. 

We can illustrate this nicely with yet another plot.

In [ ]:
predicted = logistic(fit.x,x)
plt.scatter(x, y)
plt.plot(x,predicted,color="red")
plt.xlabel('x', size=16)
plt.ylabel('y', size=16)
plt.title('Synthetic data with fitted parameters')
plt.show()  

## Curve fitting 

In this example we will be fitting an exponential curve to noisy data using scipy's ``curve_fit`` function. 

$$ f(x) = a * \mathrm{exp} \left( -bx \right) + c$$

The curve_fit function uses the Levenberg-Marquardt algorithm to perform non-linear least-square optimization.

In [ ]:
import numpy
import numpy.random
from scipy.optimize import curve_fit

def exponential(x, a, b, c):
    """Exponential function
    
    f(x) = a*exp(-b*x) + c
    
    Parameters
    ----------
    x: numpy.ndarray
        Numpy array representing range of data
        
    a, b, c: float
        Parameters of exponential function
        
    Return
    ------
    numpy array
      the output of the exponential function
    """
    return a*numpy.exp(-b*x) + c


# Create noisy dataset
x = numpy.linspace(0,4,50)
a = 2.5
b = 1.5
c = 1.0
temp = exponential(x, a, b, c) 
temp = temp + 0.1 * numpy.random.normal(size=len(temp)) 

### Curve fitting
fit_params, fit_covariances = curve_fit(exponential, x, temp)
a_fit, b_fit, c_fit = tuple(fit_params)

print("Estimated parameters a: {0:<4.2f}, b: {1:<4.2f}, c: {2:<4.2f}".format(a_fit, b_fit, c_fit))

In [ ]:
f, ax = plt.subplots(figsize=(8,8))
ax.set_ylabel('Temperature', fontsize = 16)
ax.set_xlabel('time (s)', fontsize = 16)
ax.set_xlim(0,4.1)

# Plot the data 
ax.errorbar(x, temp, fmt = 'ro', yerr = 0.1)

# Best fit curve 
ax.plot(x, exponential(x, a_fit, b_fit, c_fit),'k-', lw=2)

# Plot the +/- sigma curves
# The square root of the diagonal covariance matrix  
# element is the uncertainty on the fit parameter.
sigma_a, sigma_b, sigma_c = numpy.sqrt(fit_covariances[0,0]), \
np.sqrt(fit_covariances[1,1]), \
np.sqrt(fit_covariances[2,2])
ax.plot(x, exponential(x, a_fit + sigma_a, b_fit - sigma_b, c_fit + sigma_c), 'b-')
ax.plot(x, exponential(x, a_fit - sigma_a, b_fit + sigma_b, c_fit - sigma_c), 'b-');

# Regression using gradient descent

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generate some data
data_x = np.linspace(1.0, 10.0, 100)[:, np.newaxis]
data_y = np.sin(data_x) + 0.1 * np.power(data_x, 2) + 0.5 * np.random.randn(100, 1)

# Normalization
model_order = 6
data_x = np.power(data_x, range(model_order))
data_x /= np.max(data_x, axis=0)

# Generate test and training data set using a shuffling procedure
order = np.random.permutation(len(data_x))
portion = 20
test_x = data_x[order[:portion]]
test_y = data_y[order[:portion]]
train_x = data_x[order[portion:]]
train_y = data_y[order[portion:]]

In [ ]:
# Gradient function
def get_gradient(w, x, y):
    y_estimate = x.dot(w).flatten()
    error = (y.flatten() - y_estimate)
    mse = (1.0/len(x))*np.sum(np.power(error, 2))
    gradient = -(1.0/len(x)) * error.dot(x)
    return gradient, mse



w = np.random.randn(model_order)
alpha = 0.5
tolerance = 1e-5

# Perform Stochastic Gradient Descent
epochs = 1
decay = 0.99
batch_size = 10
iterations = 0
while True:
    order = np.random.permutation(len(train_x))
    train_x = train_x[order]
    train_y = train_y[order]
    b=0
    while b < len(train_x):
        tx = train_x[b : b+batch_size]
        ty = train_y[b : b+batch_size]
        gradient = get_gradient(w, tx, ty)[0]
        error = get_gradient(w, train_x, train_y)[1]
        w -= alpha * gradient
        iterations += 1
        b += batch_size
    
    # Keep track of our performance
    if epochs%100==0:
        new_error = get_gradient(w, train_x, train_y)[1]
        print ("Epoch: %d - Error: %.4f" %(epochs, new_error))
    
        # Stopping Condition
        if abs(new_error - error) < tolerance:
            print ("Converged.")
            break
        
    alpha = alpha * (decay ** int(epochs/1000))
    epochs += 1

print ("w =",w)
print ("Test Cost =", get_gradient(w, test_x, test_y)[1])
print ("Total iterations =", iterations)

# Plot result
y_model = np.polyval(w[::-1], np.linspace(0,1,100))
plt.plot(np.linspace(0,1,100), y_model, c='g', label='Model')
plt.scatter(train_x[:,1], train_y, c='b', label='Train Set')
plt.scatter(test_x[:,1], test_y, c='r', label='Test Set')
plt.grid()
plt.legend(loc='best')
plt.xlabel('X')
plt.ylabel('Y')
plt.xlim(0,1)
plt.show()